# 🏠 DeepRoof-2026: Multi-Task Training Notebook

Welcome to the official training environment for the **DeepRoof-2026 AI Roof Layout Engine**. 

### 🛠 Step 1: Initialize Environment
This cell resolves path issues and version conflicts. **Run this first!**

In [ ]:
import os
import sys
import subprocess
from pathlib import Path

# --- 1. SET UP PATHS ---
project_root = str(Path(os.getcwd()).parent)
if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"✅ Added {project_root} to sys.path")

venv_site = os.path.join(project_root, ".venv/lib/python3.11/site-packages")
if os.path.exists(venv_site):
    sys.path.insert(1, venv_site)
    print(f"🐍 Using venv at: {venv_site}")

# --- 2. ROBUST VERSION & DEPENDENCY CHECK ---
def get_pkg_version(package_name):
    try:
        result = subprocess.check_output([sys.executable, "-m", "pip", "show", package_name], stderr=subprocess.DEVNULL).decode()
        for line in result.split('\n'):
            if line.startswith('Version: '):
                return line.split(': ')[1].strip()
    except:
        return None
    return None

def setup_environment():
    print("🔍 Checking Environment Health...")
    requires_restart = False
    
    # Check MMCV Compatibility
    mmcv_ver = get_pkg_version("mmcv")
    if mmcv_ver:
        from mmengine.utils import digit_version
        if digit_version(mmcv_ver) >= digit_version("2.2.0"):
            print(f"🔹 Found MMCV {mmcv_ver}. Downgrading to 2.1.0 for compatibility...")
            subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "mmcv==2.1.0", "-f", "https://download.openmmlab.com/mmcv/dist/cu121/torch2.1/index.html"])
            requires_restart = True
    else:
        print("📦 MMCV missing. Installing...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "openmim"])
        subprocess.check_call([sys.executable, "-m", "mim", "install", "mmcv>=2.0.0rc4,<2.2.0"])
        requires_restart = True

    # Critical Dependencies
    for pkg in ["mmsegmentation", "ftfy", "regex", "rasterio", "geopandas", "albumentations"]:
        if not get_pkg_version(pkg):
            print(f"📦 Installing {pkg}...")
            if pkg == "mmsegmentation":
                subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "openmim"])
                subprocess.check_call([sys.executable, "-m", "mim", "install", "mmsegmentation>=1.0.0"])
            else:
                subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])
            requires_restart = True

    if requires_restart:
        print("\n" + "!"*50)
        print("CRITICAL: Environment modified. PLEASE RESTART THE KERNEL.")
        print("!"*50)
        return False
    
    print("✅ Environment is HEALTHY and COMPATIBLE.")
    return True

setup_environment()

## 📂 1. Dataset Preview

Visualize the **satellite imagery**, **instance masks**, and **surface normals**.

In [ ]:
def preview_dataset(data_root, num_samples=3):
    import matplotlib.pyplot as plt
    import numpy as np
    import cv2
    
    data_path = Path(data_root)
    if not data_path.is_absolute():
        data_path = Path(project_root) / data_root
        
    train_file = data_path / 'train.txt'
    if not train_file.exists():
        print(f"❌ Could view train.txt at {train_file}. Run prepare_omnicity_v2_final.py first!")
        return
        
    with open(train_file, 'r') as f:
        sample_ids = [line.strip() for line in f.readlines()[:num_samples]]
    
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5 * num_samples))
    for i, sid in enumerate(sample_ids):
        img_path = str(data_path / 'images' / (sid + '.jpg'))
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        
        mask_path = str(data_path / 'masks' / (sid + '.png'))
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        mask_vis = cv2.applyColorMap(((mask % 20) * 12).astype(np.uint8), cv2.COLORMAP_JET)
        
        axes[i, 0].imshow(img); axes[i, 0].set_title(f"Satellite {sid}"); axes[i, 0].axis('off')
        axes[i, 1].imshow(mask_vis); axes[i, 1].set_title("Instance Mask"); axes[i, 1].axis('off')
        
        norm_path = data_path / 'normals' / (sid + '.npy')
        if norm_path.exists():
            normals = np.load(str(norm_path))
            axes[i, 2].imshow(((normals + 1) * 127.5).astype(np.uint8))
        axes[i, 2].set_title("Surface Normals"); axes[i, 2].axis('off')
        
    plt.tight_layout(); plt.show()

preview_dataset("data/OmniCity", num_samples=2)

## ⚙️ 2. Training Configuration


In [ ]:
from mmengine.config import Config

MODE = "fine-tune" 
CONFIG_FILE = str(Path(project_root) / "configs/deeproof_finetune_swin_L.py")
WORK_DIR = str(Path(project_root) / "work_dirs/swin_l_omnicity_v2")

cfg = Config.fromfile(CONFIG_FILE)
cfg.work_dir = WORK_DIR
cfg.data_root = str(Path(project_root) / "data/OmniCity/")
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.train_cfg.max_iters = 20000

if MODE == "scratch": cfg.load_from = None
print(f"✅ Configuration Validated. WorkDir: {WORK_DIR}")

## 🚀 3. Start Training


In [ ]:
import torch
from mmengine.runner import Runner

print(f"🚀 Starting Trainer on: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

runner = Runner.from_cfg(cfg)
runner.train()